In [ ]:
!pip install opencv-python==3.4.2.16
!pip install opencv-contrib-python==3.4.2.16

ERROR: Could not find a version that satisfies the requirement opencv-python==3.4.2.16 (from versions: 3.4.2.17, 3.4.3.18, 3.4.4.19, 3.4.5.20, 3.4.6.27, 3.4.7.28, 3.4.8.29, 3.4.9.31, 3.4.9.33, 3.4.10.35, 3.4.10.37, 3.4.11.39, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 4.0.0.21, 4.0.1.23, 4.0.1.24, 4.1.0.25, 4.1.1.26, 4.1.2.30, 4.2.0.32, 4.2.0.34, 4.3.0.36, 4.3.0.38, 4.4.0.40, 4.4.0.42, 4.4.0.44, 4.4.0.46, 4.5.1.48)
ERROR: No matching distribution found for opencv-python==3.4.2.16
ERROR: Could not find a version that satisfies the requirement opencv-contrib-python==3.4.2.16 (from versions: 3.4.2.17, 3.4.3.18, 3.4.4.19, 3.4.5.20, 3.4.6.27, 3.4.7.28, 3.4.8.29, 3.4.9.31, 3.4.9.33, 3.4.10.35, 3.4.10.37, 3.4.11.39, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 4.0.0.21, 4.0.1.23, 4.0.1.24, 4.1.0.25, 4.1.1.26, 4.1.2.30, 4.2.0.32, 4.2.0.34, 4.3.0.36, 4.3.0.38, 4.4.0.40, 4.4.0.42, 4.4.0.44, 4.4.0.46, 4.5.1.48)
ERROR: No matching distribution found for opencv-contrib-python==3.4.2.16


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importation des bibliothèques nécessaires

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input 
from keras.applications.vgg16 import decode_predictions 
from keras.applications import vgg16
from keras.applications import vgg19
from keras.applications import resnet50
from keras.applications import inception_v3
from keras.applications import mobilenet
from keras.applications import xception

from skimage.transform import rotate
from skimage.feature import local_binary_pattern
from skimage import data
from skimage.color import label2rgb
from skimage.feature import greycomatrix

from matplotlib import pyplot as plt
from matplotlib.pyplot import imread
import numpy as np
import os
import cv2 
import csv
import math
import zipfile
import time

### Chargement de la base de données d'images depuis mon Google Drive

In [ ]:
files = 'GHIM' # chemin vers la base d'images

if not os.path.exists('/content/' + files):

    with zipfile.ZipFile('/content/drive/MyDrive/MIR/ghim.zip', 'r') as zip_ref:

        print('Chargement de la base de données GHIM 20...\n\n')
        zip_ref.extractall('/content')

    os.rename('GHIM-20库', files)

    print('Base de données GHIM 20 chargée ! ')
    print (len([name for name in os.listdir(files) if os.path.isfile(os.path.join(files, name))]), 'fichiers chargés')

else:

    print('Base de données GHIM 20 déjà chargée ! ')
    print (len([name for name in os.listdir(files) if os.path.isfile(os.path.join(files, name))]), 'fichiers chargés')


Chargement de la base de données GHIM 20...


Base de données GHIM 20 chargée ! 
10001 fichiers chargés


### Fonctions permettant la description d'une image par l'histogramme RGB

In [ ]:
def histogram_color(image):
    histR = cv2.calcHist([image],[0],None,[256],[0,256])
    histG = cv2.calcHist([image],[1],None,[256],[0,256])
    histB = cv2.calcHist([image],[2],None,[256],[0,256])
    hist = [histR, histG, histB]
    return hist

color = ('b','g','r')
def Cal_Histo2(image):
  hist_features = []
  if image is not None:
    des = histogram_color(image)
    hist_features.append(des)
  hist_featuresA = np.array(hist_features)
  return hist_featuresA

### Fonctions permettant la description d'une image par SIFT

In [ ]:
def siftDescriptor(image):
  sift = cv2.xfeatures2d.SIFT_create()
  gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  keypoints = sift.detect(gray,None)
  keypoints_sift, descriptor_sift = sift.compute(gray, keypoints)
  return keypoints_sift, descriptor_sift

def Cal_SIFT(image):

  if image is not None:
    kp, des = siftDescriptor(image)
  if des is not None:
    sift_features = des.tolist()
    sift_featuresA = np.array(sift_features)
  else:
    sift_featuresA = np.array([-1])
    print('None des')
  return sift_featuresA

### Fonctions permettant la description d'une image par ORB

In [ ]:
def orbDescriptor(image):
    orb = cv2.ORB()
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY) 
    keypoints, descriptor_orb = orb.detectAndCompute(gray, None)
    return keypoints, descriptor_orb

def Cal_ORB(image):
    orb = cv2.ORB_create()
    if image is not None:
      kp, des = orb.detectAndCompute(image, None)
    if des is not None:
      orb_features = des.tolist()
      orb_featuresA = np.array(orb_features)
    else:
      orb_featuresA = np.array([-1])
      print('None des')
    return orb_featuresA 

### Fonctions permettant la description d'une image par LBP

In [ ]:
def lbpDescriptor(image):
  METHOD = 'uniform'
  radius = 3
  n_points = 8 * radius
  gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  lbp = local_binary_pattern(gray, n_points, radius, METHOD)
  return lbp

def Cal_LBP(image):
  if image is not None:
    des = lbpDescriptor(image)
    lbp_features = des.tolist()
  lbp_featuresA = np.array(lbp_features)
  return lbp_featuresA

### Fonctions permettant la description d'une image par GLCM

In [ ]:
def glcm(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    glcm = greycomatrix(image, distances=[1], angles=[0, np.pi/4, np.pi/2], symmetric=True, normed=True)
    return glcm

### Fonction de sauvegarde de l'indexation sous forme d'un fichier JSON sur mon Google Drive

In [ ]:
import json


# Source : https://pynative.com/python-serialize-numpy-ndarray-into-json/
class NumpyArrayEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)


def writeJson(features, nameFileJson, exec_time, n_Images):
    imagesJson = {}
    
    imagesJson['index'] = {
        'nameDatabase' : "GHIM",
        'Descripteur' : nameFileJson,
        'exec_time' : exec_time,
        'n_Images' : n_Images
    }

    imagesJson['image'] = []

    for item in features:
          numpyData = {"array": item[1]}
          encodedNumpyData = json.dumps(numpyData, cls=NumpyArrayEncoder)

          imagesJson['image'].append({
            'data': item[0],
            'desc': encodedNumpyData,
          })

    with open("/content/drive/MyDrive/MIR/Features_train/"+nameFileJson+".json", "w") as output:
        print("Output in : \n/content/drive/MyDrive/MIR/Features_train/"+nameFileJson+".json")
        json.dump(imagesJson, output)

### Fonction d'indexation par des descripteurs de OpenCV
Renvoie la liste contenant pour chaque image son nom et ses caractéristiques calculées


In [ ]:
def indexationDesc(func, nameFileJson):
    features = [] # Liste pour stocker le nom de l'image et ses caractéristiques 

    start_time = time.time()
    pas = 0
    listPath = os.listdir(files)
    listPath.sort()

    print(f'Indexation des images de la base de données {files} avec {str(func)} : \nStockage dans le fichier {nameFileJson}.json\n')

    for j in listPath :
        data = os.path.join(files, j) 
        if not data.endswith(".jpg"):
            continue
        file_name = os.path.basename(data)
        # load an image from file
        image = cv2.imread(data)
        # describe image
        feature = func(image)
        features.append((file_name,feature))
        if pas%100 == 0:
            print (f'Nombres d\'images traitées : {pas+1}\nDernière image traitée : {file_name}\n\n')
        pas = pas + 1

    print('\nIndexation terminée, sauvegarde du fichier json')
    writeJson(features, nameFileJson, time.time()-start_time, pas)

    print('Temps d\'indexation : ', time.time() - start_time)

### Fonction d'indexation par des modèles d'ImageNet
Renvoie la liste contenant pour chaque image son nom et ses caractéristiques calculées


In [ ]:
def indexationKeras(func, nameFileJson):
    features = [] # Liste pour stocker le nom de l'image et ses caractéristiques 
    model1= func(weights='imagenet', include_top=True,pooling='avg')
    
    start_time = time.time()
    pas = 0
    listPath = os.listdir(files)
    listPath.sort()

    print(f'Indexation des images de la base de données {files} avec {func} : \nStockage dans le fichier {nameFileJson}.json\n')

    for j in listPath :
        data = os.path.join(files, j) 
        if not data.endswith(".jpg"):
            continue
        file_name = os.path.basename(data)
        # load an image from file
        if nameFileJson == "inceptionV3" or "xception":
            image = load_img(data, target_size=(299, 299))
        else : 
            image = load_img(data, target_size=(224, 224))
        # convert the image pixels to a numpy array
        image = img_to_array(image)
        # reshape data for the model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2])) # prepare the image for the VGG model
        image = preprocess_input(image)
        # predict the probability across all output classes
        feature = model1.predict(image)
        feature = np.array(feature[0]) 
        features.append((file_name,feature))
        if pas%100 == 0:
            print (f'Nombres d\'images traitées : {pas+1}\nDernière image traitée : {file_name}\n')
        pas = pas+1   

    
    print('\nIndexation terminée, sauvegarde du fichier json')
    writeJson(features,nameFileJson,time.time()-start_time, pas)

    print('Temps d\'indexation : ', time.time() - start_time)

###Lancement des indexations pour différents descripteru

In [ ]:
indexationDesc(Cal_Histo2, "HistoColor")

Indexation des images de la base de données GHIM avec <function Cal_Histo2 at 0x7f98f69c1598> : 
Stockage dans le fichier HistoColor.json

Nombres d'images traitées : 1
Dernière image traitée : 0_1.jpg


Nombres d'images traitées : 101
Dernière image traitée : 0_19.jpg


Nombres d'images traitées : 201
Dernière image traitée : 0_28.jpg


Nombres d'images traitées : 301
Dernière image traitée : 0_37.jpg


Nombres d'images traitées : 401
Dernière image traitée : 0_46.jpg


Nombres d'images traitées : 501
Dernière image traitée : 10_5001.jpg


Nombres d'images traitées : 601
Dernière image traitée : 10_5101.jpg


Nombres d'images traitées : 701
Dernière image traitée : 10_5201.jpg


Nombres d'images traitées : 801
Dernière image traitée : 10_5301.jpg


Nombres d'images traitées : 901
Dernière image traitée : 10_5401.jpg


Nombres d'images traitées : 1001
Dernière image traitée : 11_5501.jpg


Nombres d'images traitées : 1101
Dernière image traitée : 11_5601.jpg


Nombres d'images traitées

In [ ]:
'''indexationKeras(resnet50.ResNet50, "Resnet50")
indexationKeras(inception_v3, "VGG16")
indexationKeras(vgg16.VGG16, "VGG16")
indexationKeras(vgg19.VGG19, "VGG19")
indexationKerasCm1(vgg16.VGG16, "VGG16")
indexationKerasCm1(vgg19.VGG19, "VGG19")
indexationDesc(Cal_Histo2, "HistoColor")
indexationDesc(Cal_SIFT, "SIFT")
indexationDesc(Cal_ORB, "ORB")
indexationDesc(Cal_LBP, "LBP")
indexationDesc(glcm, "GLCM")'''

'indexationKeras(resnet50.ResNet50, "Resnet50")\nindexationKeras(inception_v3, "VGG16")\nindexationKeras(vgg16.VGG16, "VGG16")\nindexationKeras(vgg19.VGG19, "VGG19")\nindexationKerasCm1(vgg16.VGG16, "VGG16")\nindexationKerasCm1(vgg19.VGG19, "VGG19")\nindexationDesc(Cal_Histo2, "HistoColor")\nindexationDesc(Cal_SIFT, "SIFT")\nindexationDesc(Cal_ORB, "ORB")\nindexationDesc(Cal_LBP, "LBP")\nindexationDesc(glcm, "GLCM")'